# Exploring and Processing

## Import Statements

In [237]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import sklearn as skl
import random as rng
import nltk
import fnmatch
import docx
from lxml import etree

import docx2txt
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
from pdfminer.converter import PDFPageAggregator

from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.collocations import *
from nltk.util import ngrams
from nltk.stem.lancaster import LancasterStemmer
from string import punctuation
import sys as sys
from sys import platform
import re as re
from statistics import mode
import datefinder
from nltk import ne_chunk
from nltk.tag import StanfordNERTagger
import csv
import datetime

try:
    if(platform == "win32"):
        import win32com.client as win32
        from win32com.client import constants
except Exception as e:
    print(str(e))

import subprocess
import shutil
import glob
from docx.api import Document

In [67]:
### Test Cell for Validating Document Conversion

test_path = "C:\\fileValidation"
doc_Path = os.path.join(test_path, 'TestDocConversion.doc')
try:
    if(platform == "win32"):
        word = win32.Dispatch("Word.application")
        #word = win32.gencache.EnsureDispatch('Word.Application')
        doc = word.Documents.Open(doc_Path)
        doc.Activate()

        # Rename path with .docx
        new_file_abs = os.path.abspath(doc_Path)
        new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)

        # Save and Close
        word.ActiveDocument.SaveAs(
            new_file_abs, FileFormat=16
        )
        doc.Close(False)
        
except Exception as e:
    #doc.Close(False)
    errorFile.write(filepath + ", " + str(e) + "\n")
    print(str(e))


In [68]:
### Test Cell for Validating Document Extraction
test_Path = "C:\\fileValidation"
doc_Path = os.path.join(test_path, 'TestDocXExtraction.docx')


In [69]:
###Test Cell for Validating Path Creation
test_Path = "C:\\fileValidation"




## File Prep Functions
Functions to prepare the file and text for metadata extraction.

### Function: Is it docx or pdf?

In [70]:
def checkFileType(filename):
        if(filename.lower().endswith(('.docx'))):
            return 0
        elif(filename.lower().endswith(('.pdf'))):
            return 1
        elif(filename.lower().endswith(('.doc'))):
            return 2
        else:
            return -1

### Function: clean up text

In [72]:
def cleanText(text):
    text = text.replace("\n", ' ')
    text = text.replace("\t", ' ')
    #text = text.replace(",", ' ')
    
    dblSpacesRemaining = True
    while(dblSpacesRemaining):
        text = text.replace("  ", " ")
        if not "  " in text:
            dblSpacesRemaining = False
    return text

### Function: Process a text file

#### Process a .doc file on a Windows machine
###### Requires Microsoft Word to be installed

In [186]:
def processDocFileWindows(filepath, errorFile):
    try:
        word = win32.Dispatch("Word.application")
        doc = word.Documents.Open(filepath)
        doc.Activate()

        # Rename path with .docx
        new_file_abs = os.path.abspath(filepath)
        new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)

        # Save and Close
        word.ActiveDocument.SaveAs(
            new_file_abs, FileFormat=16
        )
        doc.Close(False)

        return new_file_abs
        
    except Exception as e:
        errorFile.write(filepath + ", " + str(e) + "\n")
        print(str(e))
        return None

#### Process a .doc file on a Mac/Linux OS
##### Uses command line function textutil

In [187]:
def processDocFileMac(filepath, errorFile):
    try:
        subprocess.run(["textutil", "-convert", "docx", filepath])
        newFilePath = re.sub(r'\.\w+$', '.docx', filepath)
        return newFilePath
    except Exception as e:
        print("Error processDocFileMac:  " + str(e))
        return None

#### Begin Processing of .doc or .docx files

In [188]:
def processDocFile(filePath, errorFile):
    
    try:
        if(platform == "win32"):
            newFilePath = processDocFileWindows(filePath, errorFile)
        else:
            newFilePath = processDocFileMac(filePath, errorFile)
        processedFilePath = processDocxFile(newFilePath, errorFile)
        if(processedFilePath):
            os.remove(filePath)
    except Exception as e:
        print("Error processDocFile:  " + str(e))
        return None
    
    
    return processedFilePath


def processDocxFile(filePath, errorFile):
    try:
        docxText = docx2txt.process(filePath)
        
        replacedText = cleanText(docxText)
        
        fileName = os.path.split(filePath)[1]
        
        baseFileName = fileName[0:-5]
        cwd = os.getcwd()
        newFilePath = os.path.join(cwd,'output',  baseFileName + ".txt")

        singleFileDocx=open(newFilePath, 'wb+')
        singleFileDocx.write(replacedText.encode("utf-8"))
        singleFileDocx.close()
        
    except Exception as e:
        print(str(e))
        return None

    return newFilePath

#### Findings:

We are using a docx specific library to process word documents, and it doesn't work on .doc files. To get around this, we found the most reliable solution (on Windows machines) was to use a Microsoft Word module to open and resave the document in the docx format. 

### Function: Process pdf file

In [217]:
def processPDFfile(filePath, errorFile):
    password = ""
    extracted_text = ""
    try:
        #print(filePath)
        fileName = os.path.split(filePath)[1]#.split('/')[-1]
        #print(fileName)
        fileName = fileName.replace(",", " ")
        baseFileName = fileName[0:-4]
    
        fp = open(filePath, "rb")
        parser = PDFParser(fp)
        document = PDFDocument(parser, password)
        
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
            
        # Create PDFResourceManager object that stores shared resources such as fonts or images
        rsrcmgr = PDFResourceManager()

        # set parameters for analysis
        laparams = LAParams()

        # Create a PDFDevice object which translates interpreted information into desired format
        # Device needs to be connected to resource manager to store shared resources
        # device = PDFDevice(rsrcmgr)
        # Extract the decive to page aggregator to get LT object elements
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)

        # Create interpreter object to process page content from PDFDocument
        # Interpreter needs to be connected to resource manager for shared resources and device 
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Ok now that we have everything to process a pdf document, lets process it page by page
        for page in PDFPage.create_pages(document):
            # As the interpreter processes the page stored in PDFDocument object
            interpreter.process_page(page)
            # The device renders the layout from interpreter
            layout = device.get_result()
            # Out of the many LT objects within layout, we are interested in LTTextBox and LTTextLine
            for lt_obj in layout:
                if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
                    newText = lt_obj.get_text()
                    newText = newText.replace('\n', ' ')
                    extracted_text += newText

        #close the pdf file
        fp.close()
        
        extracted_text = cleanText(extracted_text)#extracted_text.replace("\n", ' ')
        cwd = os.getcwd()
        newFilePath = os.path.join(cwd,'output', baseFileName + '-pdf' + ".txt")
        #print(newFilePath)
        with open(newFilePath, 'wb+') as singleFilePDF:
            singleFilePDF.write(extracted_text.encode("utf-8"))
        
    except Exception as e:
        errorFile.write(filePath + ", " + str(e) + "\n")
        print("Error processing pdf: " + str(e))
        return None
        #temp_df = processDF('./data/output/' + baseFileName + ".txt")
    return newFilePath

## NLTK  Functions
Functions involving the Natural Language Tool Kit module and concepts

### Function: DD specific text cleaning

In [76]:
def ddCleanText(text):
    newText = text.replace('Delta Dental', 'DeltaDental')
    newText = newText.replace('DELTA DENTAL', 'DELTADENTAL')
    newText = newText.replace('DeltaDental Insurance Company', 'DeltaDentalInsuranceCompany')
    return newText



### Function: Make tokenized word list

In [77]:
def getTokens(text):
    words = word_tokenize(text)
    
    customStopWords = set(stopwords.words('english')+list(punctuation))
    wordsWOStop=[word for word in words if word not in customStopWords]
    
    return wordsWOStop

### Function: Make tokenized Sentence list

In [78]:
def getSents(text):
    sents = sent_tokenize(text)

    return sents

### Function: Get Bigrams

In [79]:
def getBigrams(tokens):
    bigram_measures=nltk.collocations.BigramAssocMeasures();
    finder = BigramCollocationFinder.from_words(tokens)
    sorted_bgs = (sorted(list(finder.ngram_fd.items()), key=lambda item: item[-1], reverse=True))
    
    return sorted_bgs

### Function: Get Trigrams

In [80]:
def getTrigrams(tokens):
    trigram_measures =nltk.collocations.TrigramAssocMeasures();
    finder = TrigramCollocationFinder.from_words(tokens)
    sorted_tgs = (sorted(list(finder.ngram_fd.items()), key=lambda item: item[-1], reverse=True))

    return sorted_tgs

### Function: Get Footer

In [81]:
def getFooter(txtFile):
    try:
        if not("-pdf." in txtFile):
            docxFileName = (os.path.split(txtFile)[1]).replace(".txt",".docx")
            filePath = os.path.join(os.getcwd(),"processed",docxFileName)
            doc = docx.Document(filePath)
            footerXML = [x.blob.decode() for x in doc.part.package.parts if x.partname.find('footer')>0]

            footer = []
            for i in range(0,len(footerXML)):
                root = etree.XML(footerXML[i].split("\n",1)[1].replace("w:", ""))
                footer.append('')
                for p in root:
                    for r in p:
                    #print(r.get("t"))
                        for t in r:
                            if(t.tag == "t"):
                                footer[i] = footer[i] + t.text

            return footer
        else:
            return None
    except Exception as e:
        print(str(e))
        return None

### NLTK synonyms

#### Function: translate from syn POS to nltk POS

In [82]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return ''

#### Function: Use POS to cull wordnet synonyms

In [83]:
def getSynonyms_usingPOS(word_tuple):
    #print(word_tuple)
    word_tagged = word_tuple[0]
    word_pos = get_wordnet_pos(word_tuple[1])
    syns = wn.synsets(word_tagged, pos=word_pos)
    
    set1 = set()
    for syn in syns:
        for lem in syn.lemmas():
            set1.add(lem.name())
    #print(syns)
    return set1

#### Function: get synonyms of a single word. Helper function to Bigram and Trigram function

In [189]:
## You can't cull this one down with the POS b/c you can't tag a single word as a Part of Speech
def getSyns(word):
    syns1 = wn.synsets(word)
    
    set1 = set()
    for syn in syns1:
        for lem in syn.lemmas():
            set1.add(lem.name())
    
    return set1

#### Function: Get a similar bigram

In [85]:
def getSimilarBigrams(word1, word2):
    
    tagged_words = nltk.pos_tag([word1,word2])
    set1 = getSynonyms_usingPOS(tagged_words[0])
    if not len(set1):
        set1.add(word1)
    set2 = getSynonyms_usingPOS(tagged_words[1])
    if not len(set2):
        set2.add(word2)
    word_set = set()
    for word1 in set1:
        for word2 in set2:
            word_set.add(" ".join([word1, word2]))
    
    return word_set

#### Function: get a similar trigram

In [86]:
def getSimilarTrigrams(word1, word2, word3):
    tagged_words = nltk.pos_tag([word1,word2,word3])
    
    set1 = getSynonyms_usingPOS(tagged_words[0])
    if not len(set1):
        set1.add(word1)
    set2 = getSynonyms_usingPOS(tagged_words[1])
    if not len(set2):
        set2.add(word2)
    set3 = getSynonyms_usingPOS(tagged_words[2])
    if not len(set3):
        set3.add(word3)
    
    word_set = set()
    for word1 in set1:
        for word2 in set2:
            for word3 in set3:
                word_set.add(" ".join([word1, word2, word3]))
    #print(word_set)
    
    return word_set
        

#### Function: Get synonyms from a list of key words. Returns more keywords/phrases

In [190]:
def getSynonymsFromList(keywords):
    matches = []

    for kw in keywords:
        try:
            words = word_tokenize(kw)
        except Exception as e:
            print(str(e))
        
        if(len(words) == 1):
           
            syns = getSyns(words[0])
            for syn in syns:
                matches.append(syn)
            
        elif(len(words) == 2):
           
            syns = getSimilarBigrams(words[0],words[1])
            matches.extend(getSimilarBigrams(words[0],words[1]))
        elif(len(words) == 3):
            
            matches.extend(getSimilarTrigrams(words[0],words[1],words[2]))
        else:
            print("keyword string too long")
       
    keywords.extend(matches)
    keywords = set(keywords)

    
    return keywords

#### Findings:

Getting synonymous search terms requires a bit of filtering. If you simply pass each word into the wordnet, you get too many results, many of which have opposing definitions. One way we tried to mitigate this is by filtering on word type, which is also given by nltk. An option for future work would be to build our own corpus of words based on all of the contracts and use that to build a frequency distribution. Using this frequency distribution we could pick out synonyms that are most likely to be mentioned in a contract. 

### NLTK wrapper for Stanford NER

NLTK has a wrapper for the Stanford NER tagger
	Download the model jar file
	
The stanford NER tagger is working a bit better
http://www.nltk.org/api/nltk.tag.html#nltk.tag.stanford.StanfordTagger
https://nlp.stanford.edu/software/stanford-ner-2018-02-27.zip (download of jar files)
https://textminingonline.com/how-to-use-stanford-named-entity-recognizer-ner-in-python-nltk-and-other-programming-languages

The stanford one takes a long time to execute
	There is a faster version in CoreNLP but that's all in Java and the wrapper does not interact with it

Option: Create our own trained model from some of the files we already have and see how that does with the Stanford tagger. It could end up being faster

## MetaData
Individual metadata extraction functions as well as components for batch processing.

### Individual Metadata Functions

#### get file name
Get the filename from a full path. Determines the OS and splits the string correctly based on that

In [218]:
def getFileName(fullPath):
    try:
        if(platform == "win32"):
            fileName = fullPath.split("\\")[-1]
        else:
            fileName = fullPath.split("/")[-1]
        return fileName
    except Exception as e:
        print("Error parsing filepath: " + str(e))
        return None

#### get group number
Uses regex's made from keywords to attempt to find a group number in the file. Failing that, it searches the filename for the number.

In [219]:
def getGroupNumber(sents_tokens, filePath):
    
    group_keywords = ["group number", "groupnumber"]
    regex_exps = []
    poss_nums = []
    finalGN = None
    
    try:
        #Create regex exps out of group number keywords
        for kw in group_keywords:
                temp_re = kw + "\W\s*(?P<gn>\d+)(?P<gn2>[-\s]\d+)"
                regex = re.compile(temp_re)
                regex_exps.append(regex)

        #For each sentence, search for the expression, if found add the number to
        #list of possible group numbers
        for sent in sents_tokens:
            #print(sent)
            for my_regex in regex_exps:
                result = my_regex.search(sent.lower())
                if not result==None:
                    temp_gn = result.group('gn')
                    temp_gn2 = result.group('gn2')
                    if(temp_gn2):
                        if(len(temp_gn2[1:])>len(temp_gn)):
                            poss_nums.append(temp_gn2[1:])
                        else:
                            poss_nums.append(temp_gn)
                    else:
                        #temp_gn2 = temp_gn2[1:]
                        #print(temp_gn2)
                        
                        poss_nums.append(temp_gn)

        #Try and get the number from the file name, looking for list of numeric chars
        num_regex = re.compile("(?P<gn>\d+)(?P<gn2>[-\s]+\d+)?")
        fileName = getFileName(filePath)
        fileGN = num_regex.search(fileName)

        if not fileGN==None:#if they filename has a number sequence
            temp_gn = None
            if(fileGN.group('gn2')):
                if(len(fileGN.group('gn'))>=len(fileGN.group('gn2')[1:])):
                    temp_gn = fileGN.group('gn')
                else:
                    temp_gn = fileGN.group('gn2')[1:]

            else:
                temp_gn = fileGN.group()
            if (temp_gn in poss_nums):#then if the file group number matches one in the document, choose it
                finalGN = temp_gn
            else:
                poss_nums.append(temp_gn)#otherwise add the filename one to the list and try to get the most co
                try:
                    finalGN = mode(poss_nums)
                except Exception as e:
                    return -1
        else: #it is none and there was no group number in the filename
            try:
                finalGN = mode(poss_nums)
            except Exception as e:
                #no mode found, couldn't find a group number
                return -1
    except Exception as e:
        print("Error finding group number: " + str(e))
        return None
    
    return finalGN

#### get contract start
Uses regex and a list of keywords to attempt to find the start date of the contract. It makes multiple passes based on patterns seen in contract samples so far.

Some of the passes are necessary to filter out non-date numbers that the datefinder incorrectly parses to dates

In [221]:
def getContractStart(sents_tokens):
    start_keywords = ["effective date\S\s*\S", "effective"]
    regex_exps = []
    poss_dates = []
    finalDate = ""
    matches = []
    
    try:
        start_keywords = getSynonymsFromList(start_keywords)
        #print(start_keywords)
        for kw in start_keywords:
                temp_re = kw
                regex = re.compile(temp_re)
                regex_exps.append(regex)

    ## Original pass through sentence tokens to find possible dates
        for sent in sents_tokens:
            for my_regex in regex_exps:
                result = my_regex.search(sent.lower())
                if not result==None:
                    subset = word_tokenize(sent[sent.lower().find(result.group()):])[:20]
                    subset = " ".join(subset)
                    poss_dates.append(subset)
                    if "effective date" in subset.lower():
                        poss_dates.append(subset)

                
    ## Second pass through sentence tokens to find possible dates based on a date range format
        regex_exps = []
        backup_kw = ["\S\sthrough\s\S","\S\sthru\s\S"]
    
        for kw in backup_kw:
            temp_re = kw
            regex = re.compile(temp_re)
            regex_exps.append(regex)

        for sent in sents_tokens:
            for my_regex in regex_exps:
                result = my_regex.search(sent.lower())
                if not result==None:
                    half_1 = sent[sent.lower().find(result.group()):]
                    half_2 = sent[:sent.lower().find(result.group())]

                    subset_1 = " ".join(word_tokenize(half_1)[:6])
                    subset_2 = " ".join(word_tokenize(half_2)[-6:])
                    subset = subset_2 + subset_1

                    m = datefinder.find_dates(subset, strict=True)
                    temp_matches = []
                    for match in m:
                        if match.year >= 1966:
                            temp_matches.append(subset)
                    if len(temp_matches)>=2:
                        poss_dates.append(subset)
                        #print(subset)

                    
    ## Second pass through sentences with possible dates to eliminate ones without a year or with an invalid year
    ## These are likely other values flagged incorrectly as dates by the datefinder
    ## 1966 is the year Delta Dental was created
        for sent in poss_dates:
            #print(sent)
            find_year_re = re.compile("\d\d\d\d")
            year = find_year_re.search(sent)

            if not year==None:
                #print(sent)
                #print(year.group())
                m = datefinder.find_dates(sent, strict=True)
                for match in m:
                    if match.year >= 1966:
                        matches.append(match)
## Last pass: try to find the most common date. If there is more than one mode, choose the earliest date
##.           this seems to occur when it is finding the contract start and end in equal quantities
        #print(matches)
        try:
            finalDate = mode(matches)
        except ValueError as e:
            #print(str(e))
            if matches:
                earliestMatch = matches[0]
                for match in matches:
                    if(match < earliestMatch):
                        earliestMatch = match
                finalDate = earliestMatch
            else:
                finalDate = datetime.datetime(1066, 1, 1)
        except Exception as e:
            print("Error finding Contract start date: " + str(e))
            return None #i.e. not only could they not find a start date, something failed
    except Exception as e:
        print("Error finding Contract start date: " + str(e))
        return None
    return finalDate

##### Findings during development:

With contract start date I began by searching through the tokenized sentences with a regex expression. 
I found a datefinder module to use on each flagged sentence to pull out the dates
    Issue: the datefinder module works poorly on large, run -on sentences which are common in the contracts. It tends to find other numbers that aren't dates and try to make a date out of them.
    
    Sol: only take a subset, starting at the flagged word
    
Sometimes a match isn't found with the keywords I've seen related to the start date
    Sol: look for keywords related to contract term and take the earlier date from that sentence
    
Issue: Datefinder focusing on numbers that aren't dates
    Sol: filter for sentences that have a year (ie four digits in a row) and dates that are after Delta Dental existed (>= 1966)
    
Issue: Sometimes there are multiple modes. Usually I saw this when there were equal mentions of the end date
    Sol: if there are multiple modes, take the earliest date. 

####  get Contract End
Similar to get contract start, it uses regex and keywords over multiple passes to attempt and find the contract end.

In [223]:
def getContractEnd(sents_tokens):
    
    start_keywords = ["contract term\S\s*\S", "contract term ", "contract end", "termination date"]
    #start_keywords = ["termination date"]
    regex_exps = []
    poss_dates = []
    finalDate = ""
    matches = []
    
    try:
        start_keywords = getSynonymsFromList(start_keywords)
        #print(start_keywords)
        for kw in start_keywords:
                temp_re = kw
                regex = re.compile(temp_re)
                regex_exps.append(regex)

    ## Original pass through sentence tokens to find possible dates based on keywords
        for sent in sents_tokens:
            for my_regex in regex_exps:
                result = my_regex.search(sent.lower())
                if not result==None:
                    subset = word_tokenize(sent[sent.lower().find(result.group()):])[:30]
                    #print(subset[2])
                    if not (subset[2]=='beginning'):
                        subset = " ".join(subset)
                       # print(subset)
                        poss_dates.append(subset)


    ## Second pass through sentence tokens to find possible dates based on a date range format
        regex_exps = []
        backup_kw = ["\S\sthrough\s\S","\S\sthru\s\S", "\d\d\d\d\sto\s\S"]

        for kw in backup_kw:
            temp_re = kw
            regex = re.compile(temp_re)
            regex_exps.append(regex)

        for sent in sents_tokens:
            for my_regex in regex_exps:
                result = my_regex.search(sent.lower())
                if not result==None:
                    half_1 = sent[sent.lower().find(result.group()):]
                    half_2 = sent[:sent.lower().find(result.group())]

                    subset_1 = " ".join(word_tokenize(half_1)[:12])
                    subset_2 = " ".join(word_tokenize(half_2)[-12:])
                    subset = subset_2 + subset_1
                    #print(subset)
                    m = datefinder.find_dates(subset, strict=True)
                    temp_matches = []
                    for match in m:
                        #print(match)
                        if match.year >= 1966:
                            temp_matches.append(subset)
                    if len(temp_matches)>=2:
                        
                        poss_dates.append(subset)
                        #print(subset)


    ## Pass through sentences with possible dates to eliminate ones without a year or with an invalid year
    ## These are likely other values flagged incorrectly as dates by the datefinder
    ## 1966 is the year Delta Dental was created
        for sent in poss_dates:
            #print(sent)
            find_year_re = re.compile("\d\d\d\d")
            year = find_year_re.findall(sent)
            #print(year)
            if len(year)>=2:
                #print(year)
                m = datefinder.find_dates(sent, strict=True)
                maxMatch = datetime.datetime(1066,1,1)
                if not m == None:
                    #print(m)
                    for match in m:
                        if match > maxMatch:
                            maxMatch = match
                    if maxMatch.year >= 1966:
                        matches.append(match)
                        #print(matches)
            elif len(year)==1:
                find_year_re = re.compile("termination date")
                valid_sent = find_year_re.findall(sent.lower())
           #     print(sent)
                m = datefinder.find_dates(sent, strict=True)
                if not m == None:
          #          print(m)
                    for match in m:
                        if match.year >= 1966:
                          #  print(matches)
                            matches.append(match)

        #if(len(matches)):
        #    print(matches)

    ### If there are exactly two matches, try to find a max. If error b/c they're the same, choose one
        if(len(matches) == 2):
            try:
                finalDate = max(matches)
            except ValueError as e:
                finalDate = matches[0]
            except Exception as e:
                print("Error finding contract end date: " + str(e))
                return None

    ## If there are more, try and find the top two most mentioned and take the later. else just take the latest            
        elif(len(matches) > 2):

            try:
                date1 = mode(matches)
                matches.remove(date1)

                date2 = mode(matches)
                matches.remove(date2)

                finalDate = max([date1, date2])
            except ValueError as e:
                #print(str(e))
                if matches:
                    latestMatch = matches[0]
                    for match in matches:
                        if(match > latestMatch):
                            latestMatch = match
                    finalDate = latestMatch
            except Exception as e:
                print("Error finding Contract end date: " + str(e))
                return None
        else:
            finalDate = mode(matches)
            #return datetime.datetime(1066, 1, 1)
            #print("could not find contract end for file")
    except Exception as e:
        print("Error finding Contract end date: " + str(e))
        return None
    #print("\n")
    return finalDate

#### get Contract Duration
Uses the functions getContractStart and getContractEnd to calculate a duration if possible

In [224]:
def getContractDuration(sents_tokens):
    start = None
    end = None
    duration = None
    
    try:
        start = getContractStart(sents_tokens)
        end = getContractEnd(sents_tokens)
        
        if(start and end):
            if (start.year==1066) or (end.year == 1066):
                return -1
            else:
                duration = (end - start).days
                if duration <= 0:
                    return -1
        else:
            return -1
    except Exception as e:
        print("Error finding contract duration: " + str(e))
        return None

    return duration

#### get Client State
Attempts to find the state most often mentioned in conjunction with statements containing keywords related to the contract holder

##### Helper function to create location data set from csv

In [93]:
def makeLocationDataStruct():
    categories = []
    location_data = {}
    
    us_filename = 'us_cities_states_counties.csv'
    cwd = os.getcwd()
    filepath = os.path.join(cwd, us_filename)
    #print(filepath)
    with open(filepath, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter='|',escapechar=',')
        category_row = 1
        for row in spamreader:
            if category_row:
                for cat in row:
                    categories.append(cat)
                    location_data[cat]=set()
                category_row = 0
            else:
                
                for item in range(len(row)):
                    #print(row[item])
                    if len(row[item]):
                        location_data[categories[item]].add(row[item])
        location_data['State full'].add("Washington , DC")
        location_data['State full'].add("Washington , D.C.")
    csvfile.close()
    
    states_filename = 'state_abbrv_to_name.csv'
    filepath = os.path.join(cwd, states_filename)
    
    location_data['translate_s2l'] = {}
    location_data['translate_l2s'] = {}
    
    with open(filepath, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        category_row = 1
        for row in spamreader:
            if category_row:
                category_row = 0
            else:
                location_data['translate_s2l'][row[0]] = row[1]
                location_data['translate_l2s'][row[1]] = row[0]
    csvfile.close()
    #print(location_data)
    return location_data

##### Call function to set up location data

In [94]:
location_data = makeLocationDataStruct()


##### Get Client State Function

In [225]:
def checkIfCity(loc_str, loc_data, isDelta, isContractholder):
    cities = []
    
    if (loc_str in loc_data['City']):
        cities.append(loc_str)
    return cities

def checkIfState(loc_str, loc_data):
    states = []
    
    if(loc_str in loc_data['State full']):
        states.append(loc_str.lower())
    if(loc_str in location_data['State short']):
        try:
            states.append(location_data['translate_s2l'][loc_str].lower())
        except Exception as e:
            return states
    return states
    
def getClientLocation(sents_tokens, bgs, tgs, location_data, filename):
    loc_sents = set()
    cities = []
    states = []
    
    
    try:
        
        for sent in sents_tokens:

            sent_cities = []
            sent_states = []

            isDelta = "deltadental" in sent.lower()
            isContractholder = "contractholder" in sent.lower()
            isNotice = "notice to contractholder" in sent.lower()
            DCtext = ["washington , d.c.","washington , dc", "washington dc","district of columbia","washington d.c."]

            #Used to filter out sentences in ALL caps. They interfere with the Abbreviated States lists
            if sent.isupper():
                sent = sent.lower()


            text = nltk.word_tokenize(sent)

            i = 0

            while (i < len(text)) and (len(text) > 2):
                if(i < len(text)-2):
                    text_bg = " ".join([text[i], text[i+1]])
                else:
                    text_bg = ""
                if(i < len(text)-2):
                    text_tg = " ".join([text[i], text[i+1], text[i+2]])
                else:
                    text_tg = ""

                sent_cities.extend(checkIfCity(text[i], location_data, isDelta, isContractholder))
                sent_cities.extend(checkIfCity(text_bg, location_data, isDelta, isContractholder))
                sent_cities.extend(checkIfCity(text_tg, location_data, isDelta, isContractholder))

                sent_states.extend(checkIfState(text[i], location_data))
                sent_states.extend(checkIfState(text_bg, location_data))
                sent_states.extend(checkIfState(text_tg, location_data))

                i+=1

            if (len(sent_states)>0) and not isDelta:
                
                if bool(set(sent_states).intersection(DCtext)):
                    
                    for st in sent_states:
                        if st in DCtext:
                            states.append(st.lower())
                            if(isNotice):
                                states.append(st.lower())
                           
                short_sent_states = []
                for st in sent_states:
                    try:
                        short_sent_states.append(location_data['translate_l2s'][st.title()].lower())
                    except Exception as e:
                        e=e

                for city in sent_cities:
                    for state in sent_states+short_sent_states:
                        
                        if (len(checkIfState(city + " " + state.title(), location_data))==0) and not (city is "New York"):
                                
                                add_regex_str = str(city) + "[-,\s]+" + str(state) + "[-,\s]"
                                add_regex = re.compile(add_regex_str, re.IGNORECASE)
                                matches = add_regex.findall(sent)
                                if matches:
                                    cities.append(city)
                                    states.append(state.lower())
                                    if(isNotice):
                                        states.append(state.lower())
                                    
            
        FN_chunks = re.findall(r"[\w]+|[-\s_]", filename)
        
        for chunk in FN_chunks:
            FN_state = checkIfState(chunk, location_data)
            if len(FN_state)>0:
                for i in range(0,5):
                    states.append(FN_state[0].lower())
                    i+=1
                
              
        try:
            for st in states:
                st = st.lower()
            final_state = mode(states)
            #print(final_state)
            if(len(final_state)<3):
                try:
                    final_state = location_data['translate_s2l'][final_state.upper()].lower()
                except Exception as e:
                    #print(str(e))
                    final_state = final_state
        except ValueError as ve:
            #print(str(ve))
            if(len(states)>0):
                for st in states:
                    st = st.lower()
                try:
                    final_state = mode(states)
                except Exception as e:
                    final_state = "not_a_state"
            else:
                final_state = "not_a_state"
        except Exception as e:
            #print(str(e))
            print("Error finding Client location: " + str(e))
            final_state = None
        
    
    except Exception as e:
        print("Error finding Client location: " + str(e))
        return None #error in function execution
    return final_state

##### Findings:

Using Regular expressions to extract a full address from a file proves difficult. When looking for the client location, it is possible to narrow down the address location in the file by using keywords like 'contractholder' and avoiding keywords like 'deltadental.'

Instead of finding the full address from scratch, we moved into identifying states and cities associated with the client in the text. This worked well for states, but less so for cities, since the variety of common words that can also be city names is wide. 

We've found that working from a hardcoded list is even more effective, and enables you to get more of the address. However, unlike in the case of Delta Dental office locations, a hardcoded list of all client locations is less reasonable.

#### Delta Office Involved:
Uses regex combined with a list of possible states to find the (valid) state most often listed in conjunction with Delta Dental. This model is more effective than the client state model. The Client State function should be changed to mirror this function.

In [227]:
def getDeltaOffice(sents_tokens, bgs, tgs, location_data):
    loc_sents = set()
    states = []
    try:
        for sent in sents_tokens:
           
            sent_states = []

            isDelta = "deltadental" in sent.lower()
            isContractholder = "contractholder" in sent.lower()
            isNotice = "notice to delta dental" in sent.lower()

            if isDelta:
                
                if sent.isupper():
                    sent = sent.lower()

                #valid_states_regex = ["[wW]ashington *,? +[dD].?[cC].?","[dD]istrict [oO]f [cC]olumbia","[, ]+[gG][aA][\s,]+","[gG]eorgia","[, ]+[cC][aA][\s,]+","[cC]alifornia","[, ]+[pP][aA][\s,]+","[pP]ennsylvania","[, ]+[nN][yY][\s,]+","[nN]ew [yY]ork","[, ]+[wW][vV][\s,]+","[wW]est [vV]irginia","[, ]+[uU][tT][\s,]+","[uU]tah"]                       
                valid_states_regex = ["[, ]+(?P<state>[gG][aA])[\s,]+","(?P<state>[gG]eorgia)","[, ]+(?P<state>[cC][aA])[\s,]+","(?P<state>[cC]alifornia)","[, ]+(?P<state>[pP][aA])[\s,]+","(?P<state>[pP]ennsylvania)"]                       
                for r in valid_states_regex:
                    r = re.compile(r)
                    results = r.search(sent)
                    if(results):
                        states.append(results.group('state'))
            else:#(?P<word>\b\w+\b)
                state_and_cities_regex = ["[Aa]lpharetta[, ]+(?P<state>[gG][aA])[\s,]+","[aA]lpharetta[, ]+(?P<state>[gG]eorgia)",
                                          "[sS]acramento[, ]+(?P<state>[cC][aA])[\s,]+","[sS]acramento[, ]+(?P<state>[cC]alifornia)",
                                          "[sS]an [dD]iego[, ]+(?P<state>[cC][aA])[\s,]+","[sS]an [dD]iego[, ]+(?P<state>[cC]alifornia)",
                                          "[sS]an [fF]rancisco[, ]+(?P<state>[cC][aA])[\s,]+","[sS]an [fF]rancisco[, ]+(?P<state>[cC]alifornia)",
                                          "[mM]echanicsburg[, ]+(?P<state>[pP][aA])[\s,]+","[mM]echanicsburg[, ]+(?P<state>[pP]ennsylvania)"] 
                for r in state_and_cities_regex:
                    r = re.compile(r)
                    results = r.search(sent)
                    if(results):
                        states.append(results.group('state'))

        #print(valid_states)
        temp_states = []
        for state in states:
            if(len(state)==2):
                try:
                    long_state = location_data['translate_s2l'][state.upper()]
                    temp_states.append(long_state.lower())
                except Exception as e:
                    print(str(e))
            else:
                temp_states.append(state.lower())
        states = temp_states
        
        try:
            final_state = mode(states)
            if(len(final_state)<3):
                try:
                    final_state = location_data['translate_s2l'][final_state.upper()]
                except:
                    final_state = final_state
        except ValueError as ve:
            final_state = "not_a_state"
        except Exception as e:
            #print(e)
            print("Error finding Delta Dental office location: " + str(e))
            final_state = None
        
    except Exception as e:
        print("Error finding Delta Dental office location: " + str(e))
        final_state = None
    return final_state

#### Get contractholder name
Searches the sentences for patterns and keywords pertaining to the contractholder name, as well as looking for matches between the filename and the first few sentences of the file.

Helper function to find the longest matching substring from the filename to the sentence.

In [ ]:
def longestSubstring(filename, sent):
    filename = filename.replace("docx", "")
    filename = filename.replace("pdf", "")
    filename = filename.replace("txt", "")
    filename = re.sub(r'[-,()._]', r' ',filename)
    filename = re.sub(r'\d+', r' ', filename)
    #print(filename)
    fn_words = nltk.word_tokenize(filename.lower())
    #print(fn_words)
    
    sent = sent.replace("docx", "")
    sent = sent.replace("pdf", "")
    sent = sent.replace("txt", "")
    sent = re.sub(r'[-,()._]', r' ',sent)
    sent = re.sub(r'\d+', r' ', sent)
    sent_words = nltk.word_tokenize(sent.lower())
    matches = []
    match = []
    i=0
    j=0
    
    while(i < len(fn_words)):
        while(j < len(sent_words)):
            if fn_words[i] == sent_words[j]:
                #match found
                #match.append(fn_words[i])
                m=i
                k=j
                while((m<len(fn_words)) and (k<len(sent_words)) and (fn_words[m] == sent_words[k])):
                    match.append(fn_words[m])
                    m+=1
                    k+=1
                if(len(match)>1):
                    #re.match(r"hello[0-9]+", 'hello1')
                    file_kw = [r'schedule\s\w{1,2}', r'attachment\s[a-zA-Z]', r'appendix\s[a-zA-Z]']
                    falseMatch = 0
                    for fm in file_kw:
                        if(re.match(fm, " ".join(match))):
                            falseMatch = 1
                    if not falseMatch:
                        matches.append(match)
                match = []
                j+=1
            else:
                j+=1
        j=0
        i+=1
            
        
    longest_match = []
    if(len(matches)):
        #print(matches)
        
        for m in matches:
            m = " ".join(m)
            if(len(m) > len(longest_match)):
                longest_match = m
        
        return longest_match
    
    return None

Function searches each sentence for patterns and keywords, as well as searching the filename for the longest match among the first five sentences of the file

In [228]:
def getContractholder(sents_tokens, fileName):
    start_keywords = ["contractholder name\s?[-:]+\s+(?P<name>[\w\s]+)[gG]roup\s?[nN]umber", "contractholder\s?[-:]\s+(?P<name>[\w\s]+)[gG]roup\s?[nN]umber"]
    regex_exps = []
    poss_names = []
    finalName = ""
    matches = []
    
    try:
        for kw in start_keywords:
                temp_re = kw
                regex = re.compile(temp_re)
                regex_exps.append(regex)

        for sent in sents_tokens:
            for my_regex in regex_exps:
                result = my_regex.search(sent.lower())
                if not result==None:
                    subset = result.group('name')
                    if not (subset.startswith('the employer')):
                        poss_names.append(subset)
            
            
            
        for sent in sents_tokens[0:5]:
            substring = longestSubstring(fileName, sent)
            if substring:
                poss_names.append(substring)
        
        try:
            finalName = mode(poss_names)
        except ValueError as e:
            e=e
        except Exception as e:
            print("Error finding contractholder name: " + str(e))
            return None #i.e. not only could they not find a start date, something failed
    except Exception as e:
        print("Error finding contractholder name: " + str(e))
        return None
    return finalName

#### File type functions
Classify the file based on filename and footer information

In [192]:
def isEnterprise(fileName, footer):

    isEnterprise = 0

    enterpriseList = ["-ENT","ENT-","E-"]
    if(footer and len(footer)>0):
        for f in footer:
            if(any([x in f for x in enterpriseList])):
                isEnterprise = 1

    return isEnterprise


def isASC(filePath, footer):
    
    isASC = 0
    filename = os.path.split(filePath)[1]

    ascList = ["ASC-", "-ASC","ASO-","-ASO"," ASC", "ASC "," ASO", "ASO "]
    ascRegex = ["[- (]ASC"]
    for r in ascRegex:
        r = re.compile(r)
        results = r.search(filename)
        if(results):
            isASC = 1
    if(footer and len(footer)>0):
        for f in footer:
            if(any([x in f for x in ascList])):
                isASC = 1

    return isASC


def isEOC(filePath, footer):
    
    isEOC = 0
    filename = os.path.split(filePath)[1]

    eocList = ["EOC-", "-EOC"]
    eocRegex = ["[- (]EOC"]
    for r in eocRegex:
        r = re.compile(r)
        results = r.search(filename)
        if(results):
            isEOC = 1
    if(footer and len(footer)>0):
        for f in footer:
            if(any([x in f for x in eocList])):
                isEOC = 1
    return isEOC


def isEBB(filePath, footer):
    isEBB = 0

    filename = os.path.split(filePath)[1]


    ebbList = ["EBB-", "-EBB"]
    ebbRegex = ["[- (]EBB"]
    for r in ebbRegex:
        r = re.compile(r)
        results = r.search(filename)
        if(results):
            isEBB = 1
    if(footer and len(footer)>0):
        for f in footer:
            if(any([x in f for x in ebbList])):
                isEBB = 1

    return isEBB


def isSchedule(filePath, footer):
    isSchedule = 0

    filename = os.path.split(filePath)[1]
    #print(filename)
    schRegex = ["[sS]chedule\s+I","[ (]S[I]+[) ]","[sS]ch[ed]*[ ]+[I12]+"]
    if "schedule" in filename.lower():
        isSchedule = 1
        #print("in filename")
    else:
        for r in schRegex:
            r = re.compile(r)
            results = r.search(filename)
            if results:
                isSchedule = 1
                #print(filename)
                #print(results.group())
    #print("\n")

    return isSchedule


def isContract(filePath, footer, sentTokens):
    isContract = 0

    filename = os.path.split(filePath)[1]
    #print(filename)
    #print(sentTokens[0])
    #print("\n")
    #schRegex = ["[cC]ontract","CONTRACT"]
    contractList = ["MC-", "-MC"]
    if(footer and len(footer)>0):
        for f in footer:
            if(any([x in f for x in contractList])):
                isContract = 1

    if "contract" in filename.lower():
        isContract = 1
        #print("in filename")
        #print("\n")
    if (len(sentTokens)>5):
        for sent in sentTokens[0:5]:
            if "this contract is entered into" in sent.lower():
                isContract = 1

    return isContract


def isAttachment(filePath, footer):
    isAttachment = 0

    filename = os.path.split(filePath)[1]
    #print(filename)
    attachRegex = ["[aA]ttach[ment]*"]

    for r in attachRegex:
        r = re.compile(r)
        results = r.search(filename)
        if results:
            isAttachment = 1
            
    return isAttachment


def isAppendix(filePath, footer):
    isAppendix = 0

    filename = os.path.split(filePath)[1]
    appRegex = ["[aA]ppendix"]

    for r in appRegex:
        r = re.compile(r)
        results = r.search(filename)
        if results:
            isAppendix = 1

    return isAppendix


def isRider(filePath, footer):
    isRider = 0

    filename = os.path.split(filePath)[1]
    #print(filename)
    riderRegex = ["[-( ]R\d[\d]?"]

    for r in riderRegex:
        r = re.compile(r)
        results = r.search(filename)
        if results:
            isRider = 1
            #print(filename)
            #print(results.group())
    #print("\n")

    return isRider


def isTaxModification(filePath, footer):
    isTaxModification = 0

    filename = os.path.split(filePath)[1]
    print(filename)
    tmRegex = ["[tT]ax[- ]+[mM]odif[ication]*","TAX[- ]+MODIF[ICATION]*"]

    for r in tmRegex:
        r = re.compile(r)
        results = r.search(filename)
        if results:
            isTaxModification = 1
            #print(filename)
            print(results.group())
    print("\n")

    return isTaxModification


def isSBCModification(filePath, footer):
    isSBC = 0

    filename = os.path.split(filePath)[1]


    sbcList = ["SBC-", "-SBC"]
    sbcRegex = ["[- (_]SBC"]
    for r in sbcRegex:
        r = re.compile(r)
        results = r.search(filename)
        if(results):
            isSBC = 1
    if(footer and len(footer)>0):
        for f in footer:
            if(any([x in f for x in sbcList])):
                isSBC = 1

    return isSBC


def isPremiumAgreement(filePath, footer):
    isPremiumAgreement = 0

    filename = os.path.split(filePath)[1]


    PAList = ["PRM-", "-PRM"]
    PARegex = ["[- (]PREM AGMT", "[Pp]remium[ ]+[Aa]greement"]
    for r in PARegex:
        r = re.compile(r)
        results = r.search(filename)
        if(results):
            isPremiumAgreement = 1
    if(footer and len(footer)>0):
        for f in footer:
            if(any([x in f for x in PAList])):
                isPremiumAgreement = 1

    return isPremiumAgreement


Use all above functions to classify the input file

In [194]:
def getFileTypes(filePath, footer, sentTokens):
    types = []

    if(isEOC(filePath, footer)):
        types.append("EOC")

    if(isEBB(filePath, footer)):
        types.append("EBB")

    if(isSchedule(filePath, footer)):
        types.append("Schedule")

    if(isContract(filePath, footer, sentTokens)):
        types.append("Contract")

    if(isAttachment(filePath, footer)):
        types.append("Attachment")

    if(isRider(filePath, footer)):
        types.append("Rider")

    if(isTaxModification(filePath, footer)):
        types.append("TaxModification")

    if(isSBCModification(filePath, footer)):
        types.append("SBCModification")

    if(isPremiumAgreement(filePath, footer)):
        types.append("PremiumAgreement")
    
    if(isAppendix(filePath, footer)):
        types.append("Appendix")

    return types




### Components for batch processing to get Metadata

#### Batch pre process: Clean text and output to .txt files
Takes a folder with all of the files to be processed and creates a .txt file of the contents. This also involves the conversion from .doc to .docx

In [195]:
def batchPreProcess(errorFile, pathToData):
    cwd = os.getcwd()
    processedTextPath = ""
    
    if(os.path.isdir(pathToData)):

        for file in os.listdir(pathToData):
            filepath = os.path.join(pathToData, file)
            if(os.path.isfile(filepath)):
                print("pre-processing: " + file)
                try:
                    
                    if(checkFileType(filepath) == 0):
                        processedTextPath = processDocxFile(filepath, errorFile)
                        if not processedTextPath:
                            print("Error pre-processing file: " +  filepath)
                            
                    elif(checkFileType(filepath) == 1):
                        processedTextPath = processPDFfile(filepath, errorFile)
                        if not processedTextPath:
                            print("Error pre-processing file: " + filepath)
                            
                    elif(checkFileType(filepath) == 2):
                        processedTextPath = processDocFile(filepath, errorFile)
                        if not processedTextPath:
                            print("Error pre-processing file: " + filepath)
                            
                    else:
                        errorFile.write(filepath + ", pre-processing: invalid filetype\n")
                        raise TypeError('This path does not lead to a valid file type!')                     
                except Exception as e:
                    print(str(e))
                    errorFile.write(filepath + ", pre-processing," + str(e) + "\n")
                    print("Error pre-processing file: " + filepath)

    else:
        print("Folder data/raw doesn't exist")
        return None
    return "success"

#### Batch processing: Get basic information from .txt file
Returns file information as an array of objects containing key:value information about the file: 

In [196]:
def batchGetTokens(errorFile, dataPath):
    all_tokens = []
    cwd = os.getcwd()
    processedTextPath = ""
        
    if(os.path.isdir(dataPath)):

        for file in os.listdir(dataPath):
            filepath = os.path.join(dataPath, file)
            if(os.path.isfile(filepath) and file.endswith(".txt")):
                try:

                    temp_obj = {}

                    with open(filepath, 'r', encoding='utf-8') as txtFile:
                        text = txtFile.read()

                    temp_obj['filepath'] = filepath

                    text = ddCleanText(text)
                    temp_obj['cleanText'] = text

                    wordTokens = getTokens(text)
                    sentTokens = getSents(text)
                    temp_obj['wordTokens'] = wordTokens
                    temp_obj['sentTokens'] = sentTokens

                    bgs = getBigrams(wordTokens)
                    tgs = getTrigrams(wordTokens)
                    temp_obj['bgs'] = bgs
                    temp_obj['tgs'] = tgs

                    temp_obj['footer'] = getFooter(filepath)
                    txtFile.close()
                    all_tokens.append(temp_obj)
                except Exception as e:
                    print(str(e))


    else:
        print("Folder /output doesn't exist. Pre-processing failed.")
        return None
    return all_tokens


#### get metadata attributes
This function takes in a single files info and calls all of the metadata functions

In [197]:
def getMetaDataAtt(file_info):
    #print(file_info)
    file_attr = {}
    file_attr['filepath'] = file_info['filepath']

    fileName = getFileName(file_info['filepath'])
    if not fileName:
        fileName = os.path.split(file_info['filepath'])[1]
        file_attr['fileName'] = fileName
    else:
        #print(fileName)
        file_attr['fileName'] = fileName

    fileType = isEnterprise(fileName, file_info['footer'])
    file_attr['fileType'] = fileType

    groupNumber = getGroupNumber(file_info['sentTokens'], file_info['filepath'])
    file_attr['groupNumber'] = groupNumber


    contractStartDate = getContractStart(file_info['sentTokens'])
    file_attr['contractStartDate'] = contractStartDate


    contractEndDate = getContractEnd(file_info['sentTokens'])
    file_attr['contractEndDate'] = contractEndDate


    contractDuration = getContractDuration(file_info['sentTokens'])
    file_attr['contractDuration'] = contractDuration

    
    clientLocation = getClientLocation(file_info['sentTokens'], file_info['bgs'], file_info['tgs'], location_data, fileName)
    clientLocation = clientLocation.replace(",","")
    file_attr['clientLocation'] = clientLocation


    deltaOfficeLocation = getDeltaOffice(file_info['sentTokens'], file_info['bgs'], file_info['tgs'], location_data)
    deltaOfficeLocation = deltaOfficeLocation.replace(",","")
    file_attr['deltaOfficeLocation'] = deltaOfficeLocation


    contractholderName = getContractholder(file_info['sentTokens'], fileName)
    file_attr['contractholderName'] = contractholderName

    ent = isEnterprise(file_info['filepath'], file_info['footer'])
    file_attr['isENT'] = ent

    asc = isASC(file_info['filepath'], file_info['footer'])
    file_attr['isASC'] = asc

    fileTypes = getFileTypes(file_info['filepath'], file_info['footer'], file_info['sentTokens'])
    typeRanks = ["Contract","EOC","Attachment","Schedule","EBB","Appendix","Rider","TaxModification","SBCModification","PremiumAgreement"]
    mainFileType = ""
    for rank in typeRanks:
        if rank in fileTypes:
            mainFileType = rank
            break
    file_attr['mainFileType'] = mainFileType
    file_attr['fileTypes'] = fileTypes


    file_attr['footer'] = file_info['footer']
    eoc = isEOC(file_info['filepath'], file_info['footer'])
    file_attr['isEOC'] = eoc


    return file_attr



## Workspace Prep Functions
These functions set up the workspace if it does not yet exist. These must be run first before calling any processing functions.

### Function: Initial setup
    Just sets up the expected folder structure. Fills nothing.

In [229]:
def setupWorkspace(rawPath, errorFile):
    cwd = os.getcwd()
    print("cwd: " + cwd)
    
    
    processedPath = os.path.join(cwd, "processed")
    outputPath = os.path.join(cwd, "output")
    dataPath = os.path.join(cwd, "data")
    
    try:
        if not(os.path.isdir(rawPath)):
            raise Exception(rawPath + " doesn't exist.")
        elif not(os.listdir(rawPath)):
            raise Exception(rawPath + " is empty.")
        else:
        
            if(os.path.isdir(processedPath)):
                raise Exception(processedPath + " already exists.")
            else:
                print("creating " + processedPath + "...") 
                os.makedirs(processedPath)

            if(os.path.isdir(outputPath)):
                raise Exception(outputPath + " already exists.")
            else:
                print("creating " + outputPath + "...")
                os.makedirs(outputPath)

            if(os.path.isdir(dataPath)):
                raise Exception(dataPath + " already exists.")
            else:
                print("creating " + dataPath + "...")
                os.makedirs(dataPath)
            return "Success"
    except Exception as e:
        print("Error setting up environment: " + str(e))
        return None   

### Function: Move to processed folder
    Moves all files in the raw folder to the processed folder

In [230]:
def moveToProcessedFolder(rawPath, errorFile):
    cwd = os.getcwd()
    print("cwd: " + cwd)
    
    processedPath = os.path.join(cwd, "processed")
    outputPath = os.path.join(cwd, "output")
    dataPath = os.path.join(cwd, "data")
    
    try:
        if(os.path.isdir(rawPath) and os.path.isdir(processedPath)):
            #is it empty?
            if not os.listdir(rawPath):
                raise Exception("raw data folder is empty.")
            else:
                for file in os.listdir(rawPath):
                    print("processing: " + file)
                    #move a copy to processed Path
                    try:
                        #print("moving: " + file)
                        shutil.copy(os.path.join(rawPath, file), os.path.join(processedPath, file))
                    #catch copy exception here so it doesn't stop all files (?)
                    except Exception as e:
                        print("Error copying file to processed folder: " + str(e))
                return processedPath
                        
            
        else:
            raise Exception("Expected file structure doesn't exist.")
            
        
    except Exception as e:
        print("Error filling processed folder: " + str(e))
        return None

### Function: Fill output folder with .txt files
    Process all files in the Processed folder and output resulting txt to output folder

In [231]:
def createTxtFiles(pathToData, errorFile):
    cwd = os.getcwd()
    processedTextPath = ""
    
    #dataPath = os.path.join(cwd, "processed")
    if(os.path.isdir(pathToData)):

        for file in os.listdir(pathToData):
            filepath = os.path.join(pathToData, file)
            if(os.path.isfile(filepath)):
                print("pre-processing: " + file)
                try:
                    
                    if(checkFileType(filepath) == 0):
                        processedTextPath = processDocxFile(filepath, errorFile)
                        if not processedTextPath:
                            print("Error pre-processing file: " +  filepath)
                            
                    elif(checkFileType(filepath) == 1):
                        processedTextPath = processPDFfile(filepath, errorFile)
                        if not processedTextPath:
                            print("Error pre-processing file: " + filepath)
                            
                    elif(checkFileType(filepath) == 2):
                        processedTextPath = processDocFile(filepath, errorFile)
                        if not processedTextPath:
                            print("Error pre-processing file: " + filepath)
                            
                    else:
                        #errorFile.write(filepath + ", pre-processing: invalid filetype\n")
                        raise TypeError('This path does not lead to a valid file type!')                     
                except Exception as e:
                    print("Error pre-processing: " + str(e))
                    #errorFile.write(filepath + ", pre-processing," + str(e) + "\n")
                    print("Error pre-processing file: " + filepath)

    else:
        print("Folder data/raw doesn't exist")
        return None
    return "success"

### Run it

In [106]:
def rawToWorkspace(pathToRawData):
    errorFilePath = os.path.join(os.getcwd(),'cannot_process.csv')
    errorFile = open(errorFilePath, 'w')
    
    setup = setupWorkspace(pathToRawData,errorFile)
    if(setup):
        moveToProcessed = moveToProcessedFolder(pathToRawData,errorFile)
        if(moveToProcessed):
            createTxt = createTxtFiles(os.path.join(os.getcwd(), "processed"),errorFile)
            if(createTxt):
                errorFile.close()
                return "Success"
    errorFile.close()
    return None
    
#rawToWorkspace("C:\\Users\\Sydney.knox\\Documents\\rawDataDI")

## Data by Group Number
These functions explored taking the information found by the metadata attributes and deviding the files into a folder structure by: Group Number --> Contract Holder Name --> Singular Contract

These functions are not currently being utilized in the final workflow, but are functional

### Function: Create Group folders and move group files in

In [232]:
def getGroupFolders():
    cwd = os.getcwd()
    dataPath = os.path.join(cwd, "data")
    outputPath = os.path.join(cwd, "output")
    
    errorFilePath = os.path.join(os.getcwd(), 'cannot_process.csv')
    errorFile = open(errorFilePath, 'w')
    
    try:
        if not(os.path.isdir(dataPath)):
            raise Exception(dataPath + " doesn't exist.")
        elif not(os.path.isdir(outputPath)):
            raise Exception(outputPath + " doesn't exist.")
        elif not(os.listdir(outputPath)):
            raise Exception(dataPath + " is empty.")
        else:
            #get all of the base info for each file
            base_info = batchGetTokens(errorFile, outputPath)
            if not base_info:
                print("Error in getting tokens")
            else:
                
                sorted_file_info = {}
                #print(sorted_file_info)
                #Get all the meta data for each file
                for file in base_info:
                    destPath = dataPath
                    
                    file_attr = getMetaDataAtt(file)
                    if((file_attr['groupNumber']) and not(int(file_attr['groupNumber']) is -1)):
                        gn = file_attr['groupNumber']
                        destPath = os.path.join(destPath, str(int(gn)))
                    else:
                        gn = -1
                    #destPath = os.path.join(destPath, "no_group_number")
                    try:
                        sorted_file_info[str(int(gn))].append(file_attr)
                    except KeyError as e:
                        sorted_file_info[str(int(gn))] = []
                        sorted_file_info[str(int(gn))].append(file_attr)
                    except Exception as e:
                        print("Error finding group number folder for file: " + str(e))

                for group in sorted_file_info:
                    print(group)
                    group_start_dates = set()
                    group_ch_names = set()
                    oneStartDate = 0
                    oneCHName = 0
                    
                    
                    for file in sorted_file_info[group]:
                        if(file['contractStartDate'].year > 1950):
                            group_start_dates.add(file['contractStartDate'].date())
                        if(len(file['contractholderName'])>0):
                            group_ch_names.add(file['contractholderName'].rstrip())
                
                
                    if(len(group_start_dates)==1):
                        oneStartDate = 1
                    elif(len(group_start_dates)==0):
                        oneStartDate = 1
                        group_start_dates.add("unknown_startDate")
                    if(len(group_ch_names)==1):
                        oneCHName = 1
                    elif(len(group_ch_names)==0):
                        oneCHName = 1
                        group_ch_names.add("unknown_name")

                    for file in sorted_file_info[group]:
                        if(int(group) == -1):
                            destPath = os.path.join(dataPath, "no_group_number")
                        else:
                            destPath = os.path.join(dataPath, str(int(group)))
                        #print(destPath)
                        if oneCHName:
                            temp = group_ch_names.pop()
                            destPath = os.path.join(destPath, temp)
                            #print(destPath)
                            group_ch_names.add(temp)
                        else:
                            if(len(file['contractholderName'])>0):
                                destPath = os.path.join(destPath, file['contractholderName'])
                            else:
                                destPath = os.path.join(destPath, "unknown_name")
                        if oneStartDate:
                            temp = group_start_dates.pop()
                            destPath = os.path.join(destPath, str(temp))
                            #print(destPath)
                            group_start_dates.add(temp)
                        else:
                            destPath = os.path.join(destPath, str(file['contractStartDate'].date()))
                        
                        try:
                            print(destPath)
                            os.makedirs(destPath, exist_ok=True)
                            #print(file['filepath'])
                            shutil.copy(file['filepath'], destPath)
                        except Exception as e:
                            print("Error creating folder: " + str(e))
        
                    
    except Exception as e:
        print("Error organizing data into group and contract folders: " + str(e))


### Run 'em

In [108]:
#getGroupFolders()

### Collect information and output a csv using group folders

In [233]:
def createGroupCSV():
    cwd = os.getcwd()
    dataPath = os.path.join(cwd, "data")

    errorFilePath = os.path.join(os.getcwd(),'data','cannot_process.csv')
    errorFile = open(errorFilePath, 'w')

    #attrByGroupFilePath = os.path.join(os.getcwd(), 'data', 'raw_attr_data_byGroup.csv')
    #attrByGroupFile = open(attrByGroupFilePath, 'w')

    try:
        if(os.path.isdir(dataPath)):

            for group in os.listdir(dataPath):
                print(group)            
                groupPath = os.path.join(dataPath, group)
                if(os.path.isdir(groupPath)):
                    base_info = batchGetTokens(errorFile, groupPath)
                    if not base_info:
                        print("Error in getting tokens")
                    else:
                        outputFilePath = os.path.join(groupPath,'group_'+ group + '_attribute_data.csv')
                        outputFile=open(outputFilePath, 'w')    
                        first_row = 1

                        for file in base_info:
                            file_attr = getMetaDataAtt(file)
                            if first_row:
                                for key in file_attr:
                                    outputFile.write(key + ",")
                                outputFile.write("\n")
                                first_row = 0
                            else:
                                for attr in file_attr:
                                    outputFile.write(str(file_attr[attr])+ ",")
                            outputFile.write("\n")

                        outputFile.close()

        else:
            raise Exception("data folder doesn't exist.")
    except Exception as e:
        print("Error printing individual group attributes csv: " + str(e))

    errorFile.close()
    
    
def collectGroupInfo():
    cwd = os.getcwd()
    dataPath = os.path.join(cwd, "data")

    errorFilePath = os.path.join(os.getcwd(),'data','cannot_process.csv')
    errorFile = open(errorFilePath, 'w')

    attrByGroupFilePath = os.path.join(os.getcwd(), 'data', 'raw_attr_data_byGroup.csv')
    attrByGroupFile = open(attrByGroupFilePath, 'w')

    try:
        if(os.path.isdir(dataPath)):

            for group in os.listdir(dataPath):
                
                
                
                groupPath = os.path.join(dataPath, group)
                if(os.path.isdir(groupPath)):
                    print(group)
                    group_attr = {}
                    first_file = 1
                    
                    base_info = batchGetTokens(errorFile, groupPath)
                    if not base_info:
                        print("Error in getting tokens")
                    else:
                        
                        for file in base_info:
                            file_attr = getMetaDataAtt(file)
                            #print(file_attr)
                            if first_file:
                                for key in file_attr:
                                    try:
                                        group_attr[key] = []
                                    except Exception as e:
                                        print(str(e))
                                first_file = 0
                            
                            print(group_attr)
                            
                            for attr in file_attr:
                                #print(attr)
                                #print(file_attr[attr] == "-1")
                                #print(attr)
                                #print((attr is 'groupNumber') or (attr is 'contractDuration'))
                                if((attr is 'groupNumber') or (attr is 'contractDuration')):
                                    #print(file_attr[attr] == "-1")
                                    #print(file_attr[attr] == -1)
                                    if not ((file_attr[attr]) == -1):
                                        group_attr[attr].append(file_attr[attr])
                                if((attr is 'contractStartDate') or (attr is 'contractEndDate')):
                                    #print((file_attr[attr] == datetime.datetime(1066,1,1)))
                                    if not (file_attr[attr] == datetime.datetime(1066,1,1)):
                                        group_attr[attr].append(file_attr[attr])
                                if((attr is 'clientLocation') or (attr is 'deltaOfficeLocation')):
                                    #print(file_attr[attr] is 'not_a_state')
                                    if not file_attr[attr] is 'not_a_state':
                                        group_attr[attr].append(file_attr[attr])
                                if(attr is 'contractHolderName'):
                                    if not file_attr[attr] is '':
                                        group_attr[attr].append(file_attr[attr])
                            print(group_attr)
                        #attrByGroupFile.write("\n")
                        for attr in group_attr:
                            #print(group_attr[attr])
                            try:
                                attrMode = mode(group_attr[attr])
                                attrByGroupFile.write(str(attrMode) + ",")
                                print(attrMode)
                            except ValueError as ve:
                                if(len(group_attr[attr])>0):
                                    attrByGroupFile.write(str(set(group_attr[attr])) + ",")
                                else:
                                    attrByGroupFile.write(",")
                            except Exception as e:
                                #print(attr + "has no mode.")
                                attrByGroupFile.write(",")
                                print("Error with attr " + str(attr) + ": " + str(e))
                        attrByGroupFile.write("\n")
                            

        else:
            raise Exception("data folder doesn't exist.")
    except Exception as e:
        print("Error creating consolidated group csv: " + str(e))

    errorFile.close()
    attrByGroupFile.close()

In [110]:
#collectGroupInfo()

## Rate Table Extraction
All functions for extracting the rate tables and outputting the information to csv files.

#### Define if table object is a Rate table

In [198]:
def isRateTable(table):
    count = 0
    paymentFlag = 0
    percentFind = re.compile("%")
    for row in table.rows:
        for cell in row.cells:
            result = re.findall(percentFind, cell.text)
            if(len(result)):
                count += len(result)
            if(((cell.text).lower()).find('contractholder shall pay') > -1) or (((cell.text).lower()).find('primary enrollee shall pay') > -1):
                paymentFlag = 1
    if (count >= 2) and (paymentFlag == 0):
        return True
    else:
        return False

#### Define if table object is a Multi-plan table

In [199]:
def isMultiPlan(table):
    isMultiPlan = False
    multiPlanList = ["high plan", "enhanced plan", "plan design"]
    for row in table.rows:
        for cell in row.cells:
            if(any([x in (cell.text).lower() for x in multiPlanList])):
                isMultiPlan = True
    return isMultiPlan

#### Process a table object identified as a normalized, single plan

In [200]:
def processSinglePlan(table):
    first_row=1
    first_row_found = 0
    title_row=1
    temp_obj = []
    row_template = []

    for row in table.rows:

        if(((row.cells[0]).text).lower().find("contract benefit level") > -1):
            first_row_found = 1
        else:
            if first_row_found:

                if(title_row):
                    row_template = []
                    for cell in row.cells:
                        row_template.append(cell.text)
                    title_row = 0
                else:
                    new_row = {}
                    for cat in row_template:
                        new_row[cat] = ""
                    for index, cell in enumerate(row.cells):
                        new_row[row_template[index]] = cell.text

                    temp_obj.append(new_row)

    if not first_row_found:#never found contract benefit levels
         for row in table.rows:
            ppoFlag = 0
            non_ppoFlag = 0

            ppoList = ["ppo providers", "pposm providers", "dpo providers", "delta dental ppo", "in-network"]
            non_ppoList = ["non-delta dental providers","out-of-network"]

            for cell in row.cells:
                #print(cell.text)
                if any([x in (cell.text).lower() for x in ppoList]):
                    ppoFlag = 1
                    #print(cell.text)
                if any([x in (cell.text).lower() for x in non_ppoList]):
                    non_ppoFlag = 1
                    #print(cell.text)

            if(ppoFlag and non_ppoFlag):
                first_row_found = 1

            if first_row_found:
                if(title_row):
                    row_template = []
                    for cell in row.cells:
                        row_template.append(cell.text)
                    title_row = 0
                    #print(row_template)
                else:
                    new_row = {}
                    for cat in row_template:
                        new_row[cat] = ""
                    for index, cell in enumerate(row.cells):
                        new_row[row_template[index]] = cell.text

                    #print(new_row)
                    temp_obj.append(new_row)

    #print(temp_obj)
    #print("\n")

    plan_obj = []

    for row in temp_obj:
        cat = {}
        categoryIdentifiers = ["categor","benefits"]
        for cell in row:
            if (len(cell)==0) or (any([x in cell.lower() for x in categoryIdentifiers])):
                if ("diagnostic" in (row[cell]).lower()) and ("preventive" in (row[cell]).lower()):
                    cat['category'] = "DandP"
                elif ("basic" in (row[cell]).lower()):
                    cat['category'] = "basic"
                elif ("major" in (row[cell]).lower()):
                    cat['category'] = "major"
                elif ("orthodontic" in (row[cell]).lower()):
                    cat['category'] = "orthodontic"

                else:
                    if(len(row)>=3):
                        cat['category'] = row[cell].lower().replace("\n"," ")

            elif(" ppo" in cell.lower()) or ("in-network" in cell.lower()) or (" dpo " in cell.lower()):
                kw = ["%","not covered"]

                if any([x in (row[cell]).lower() for x in kw]):
                    cat['PPO_rate'] = row[cell]

            elif("non-delta" in cell.lower()) or ("out-of-network" in cell.lower()):
                kw = ["%","not covered"]

                if any([x in (row[cell]).lower() for x in kw]):
                    cat['non-PPO_rate'] = row[cell]

        if(len(cat)>=3):
            plan_obj.append(cat)

    return plan_obj

#### Process a table object identified as a normalized with multiple plans

In [201]:
def processMultiPlan(table):
    first_row=1
    first_row_found = 0
    plan_row=1
    title_row=1
    temp_obj = {}
    row_template = []

    for row in table.rows:
        if not first_row_found:
            for cell in row.cells:
                if((cell.text).lower().find("contract benefit level") == 0):
                    first_row_found = 1
        else:
            if(plan_row):
                plan_template = []
                for cell in row.cells:
                    plan_template.append(cell.text)
                plan_row = 0
               # print(plan_template)
                for plan in plan_template:
                    temp_obj[plan] = []
               # print(temp_obj)
            elif(title_row):
                row_template = []
                for cell in row.cells:
                    row_template.append(cell.text)
                title_row = 0
                #print(row_template)
            else:
                for plan in temp_obj:
                    if ("plan" in plan.lower()) and not ("delta dental will pay" in (row.cells[0].text).lower()):
                       # print(plan)
                        new_row = {}
                        for index, cat in enumerate(row_template):
                            if not("plan" in plan_template[index].lower()):
                                new_row[cat] = ""
                            if(plan_template[index] == plan):
                                new_row[cat] = ""
                        for index, cell in enumerate(row.cells):
                            if not("plan" in plan_template[index].lower()):
                                new_row[row_template[index]] = cell.text
                            if(plan_template[index] == plan):
                                new_row[row_template[index]] = cell.text
                        #print(new_row)
                        temp_obj[plan].append(new_row)

    if not first_row_found:#never found contract benefit levels
        highPlanFlag = 0
        lowPlanFlag = 0
        for row in table.rows:
            #ppoFlag = 0
            #non_ppoFlag = 0
            if(len(set(row.cells))==1):
                #print(row)
                break

            #ppoList = ["ppo providers", "pposm providers", "dpo providers", "delta dental ppo", "in-network"]
            #non_ppoList = ["non-delta dental providers","out-of-network"]
            highPlanList = ["high plan", "enhanced plan", "premier plan"]
            lowPlanList = ["low plan", "standard plan", "ppo plan"]

            for cell in row.cells:
                #print(cell.text)
                if any([x in (cell.text).lower() for x in highPlanList]):
                    highPlanFlag = 1
                    #print(cell.text)
                if any([x in (cell.text).lower() for x in lowPlanList]):
                    lowPlanFlag = 1
                    #print(cell.text)

            if(highPlanFlag and lowPlanFlag):
                if(plan_row):
                    plan_template = []
                    for cell in row.cells:
                        plan_template.append(cell.text)
                    plan_row = 0
                   # print(plan_template)
                    for plan in plan_template:
                        temp_obj[plan] = []
                   # print(temp_obj)
                elif(title_row):
                    row_template = []
                    for cell in row.cells:
                        row_template.append(cell.text)
                    title_row = 0
                    #print(row_template)
                else:
                    for plan in temp_obj:
                        if ("plan" in plan.lower()) and not ("delta dental will pay" in (row.cells[0].text).lower()):
                           # print(plan)
                            new_row = {}
                            for index, cat in enumerate(row_template):
                                if not("plan" in plan_template[index].lower()):
                                    new_row[cat] = ""
                                if(plan_template[index] == plan):
                                    new_row[cat] = ""
                            for index, cell in enumerate(row.cells):
                                if not("plan" in plan_template[index].lower()):
                                    new_row[row_template[index]] = cell.text
                                if(plan_template[index] == plan):
                                    new_row[row_template[index]] = cell.text
                            #print(new_row)
                            temp_obj[plan].append(new_row)

    #print(temp_obj)
    plan_set = {}

    for plan in temp_obj:
        #print(plan)
        plan_obj = []
        for row in temp_obj[plan]:
            #print(row)
            cat = {}
            for cell in row:
                if(" ppo" in cell.lower()) or ("in-network" in cell.lower()) or (" dpo " in cell.lower()):
                    kw = ["%","not covered","n/a"]

                    if any([x in (row[cell]).lower() for x in kw]):
                        cat['PPO_rate'] = row[cell]
                elif("non-delta" in cell.lower()) or ("out-of-network" in cell.lower()):
                    kw = ["%","not covered","n/a"]

                    if any([x in (row[cell]).lower() for x in kw]):
                        cat['non-PPO_rate'] = row[cell]

                else:#("categor" in cell.lower()) or (len(cell)==0):
                    #print(cell.lower())
                    #print((row[cell]).lower())
                    if ("diagnostic" in (row[cell]).lower()) and ("preventive" in (row[cell]).lower()):
                        cat['category'] = "DandP"
                    elif ("basic benefit" in (row[cell]).lower()) or ("basic service" in (row[cell]).lower()):
                        cat['category'] = "basic"
                    elif ("major benefit" in (row[cell]).lower()) or ("major service" in (row[cell]).lower()):
                        cat['category'] = "major"
                    elif ("orthodontic benefit" in (row[cell]).lower()) or ("orthodontic service" in (row[cell]).lower()):
                        cat['category'] = "orthodontic"

                    else:
                        cat['category'] = (row[cell]).lower().replace("\n"," ")

            if(len(cat)>=3):
                plan_obj.append(cat)
        if(len(plan_obj)>0):
            plan_set[plan] = plan_obj

    #print(plan_set)
    return plan_set

#### Define if a table object represents a normalized plan

In [202]:
def isNormalized(table):
    isNormalized = False
    for row in table.rows:
        if(((row.cells[0].text).lower()).find('contract benefit levels')):
            isNormalized = True  
    return isNormalized

#### Define if a table object represents a denormalized plan

In [234]:
def isDenormalized(sentTokens):
    isDeNormalized = False
    BCSfound = False

    for index, sent in enumerate(sentTokens):
        if("benefit summary chart" in sent.lower()):
            count = 0
            regex = re.compile("%")
            for i in range(0,20):
                try:
                    numPercents = regex.findall(sentTokens[index + i])
                    #print(len(numPercents))
                    count += len(numPercents)
                except Exception as e:
                    print("Error determining normalization: " + str(e))
            #print(count)
            if(count > 20):
                    isDeNormalized = 1
    return isDeNormalized

#### Standardize the wording in a plan object
The wording in the rate tables is diverse, this standardizes the data points to allow analysis

In [209]:
def standardizePlanObject(plan_obj, outputFile, row):
    
    categoryList = ['DandP','basic','major','orthodontic']
    try:
        for cat in categoryList:
            found=0
            for plan_row in plan_obj:
                if (len(plan_row)>=3) and (plan_row['category'] == cat):
                    found = 1
                    row[str(cat)+" PPO"] = plan_row['PPO_rate']
                    row[str(cat)+" non-PPO"] = plan_row['non-PPO_rate']

        for plan_row in plan_obj:
            if (plan_row['category'] not in categoryList) and (len(plan_row)>=3 and plan_row['PPO_rate'] and plan_row['non-PPO_rate']):
                row[str(plan_row['category']) + " PPO"] = plan_row['PPO_rate']
                row[str(plan_row['category']) + " non-PPO"] = plan_row['non-PPO_rate']

    except Exception as e:
        print(str(e))
        print(row['Filename'])
        print(plan_obj)

    return row


#### Extract all rate tables from a DocX file

In [235]:
def getRateTables(file_info):
    plan_objs = []
    try:
        filePath = file_info['filepath']
        fileName = file_info['fileName']

        if not "-pdf." in fileName:
            docxFileName = fileName.replace(".txt", ".docx")
            docxFilePath = os.path.join(os.getcwd(), "processed", docxFileName)

            document = docx.Document(docxFilePath)
            docRateTables = []

            for table in document.tables:
                if(isRateTable(table)):
                    if(isMultiPlan(table)):
                        plan_objs = processMultiPlan(table)
                    else:
                        plan_obj = processSinglePlan(table)
                        plan_objs.append(plan_obj)
    except Exception as e:
        print("Error extracting rate tables: " + str(e))
        return None
    return plan_objs 

#### Create object of table_rows to be written to the comprehensive Rate output csv

In [236]:
def rateTableCSV():
    from docx import Document
    cwd = os.getcwd()

    errorFilePath = os.path.join(os.getcwd(),'data','cannot_process.csv')
    errorFile = open(errorFilePath, 'w')

    base_info = None
    #pp = batchPreProcess(errorFile, os.path.join(cwd, "processed"))
    #if pp == None:
    #    print("Error in pp")
    #else:
    dataPath = os.path.join(cwd, "output")
    base_info = batchGetTokens(errorFile, dataPath)
    if not base_info:
        print("Error in getting tokens")
    else:
        outputFilePath = os.path.join(os.getcwd(),'rate_table_output.csv')
        outputFile=open(outputFilePath, 'w')
        #first_row = 1
        table_rows = []
        #outputFile.write("Filename,hasRateTable,isMultiplan,isNormalized,DandP PPO,DandP non-PPO,basic PPO,basic non-PPO,major PPO,major non-PPO,orthodontics PPO,orthodontics non-PPO\n")

        for file in base_info:#os.listdir(os.path.join(cwd, "processed")):
            row_template = {
                        "Filename":"",
                        "hasRateTable":"",
                        "isMultiplan":"",
                        "isNormalized":"",
                        "DandP PPO":"",
                        "DandP non-PPO":"",
                        "basic PPO":"",
                        "basic non-PPO":"",
                        "major PPO":"",
                        "major non-PPO":"",
                        "orthodontic PPO":"",
                        "orthodontic non-PPO":""
                    }
            row = row_template
            try:
                #file['fileName'] = os.path.split(file['filepath'])[1]
                row['Filename'] = os.path.split(file['filepath'])[1]
                if(isDenormalized(file['sentTokens'])):
                    #outputFile.write("\""+file['fileName'] + "\"" + ",True,False,False,\n")
                    row['hasRateTable'] = "True"
                    row['isMultiplan'] = "False"
                    row['isNormalized'] = "False"
                    table_rows.append(row)
                elif not(row['Filename'].endswith("-pdf.txt")):
                    docxFileName = row['Filename'].replace("txt","docx")
                    document = Document(os.path.join(cwd, "processed", docxFileName))
                    docRateTables = []
                    #print(docxFileName)#outputFile.write(file+",")
                    if(len(document.tables)>0):
                        rateTableFound = 0
                        for table in document.tables:
                            if(isRateTable(table)):
                                rateTableFound = 1
                                #print("is rate table")
                                if(isMultiPlan(table)):
                                    #print("is Multiplan\n")
                                    list_plan_objs = processMultiPlan(table)
                                    #print(len(list_plan_objs))
                                    for plan_obj in list_plan_objs:
                                        row = row_template
                                        row['Filename'] = os.path.split(file['filepath'])[1]
                                        #outputFile.write("\"" + docxFileName + "\"" + ",True,True,True,")
                                        row['hasRateTable'] = "True"
                                        row['isMultiplan'] = "True"
                                        row['isNormalized'] = "True"
                                        row = standardizePlanObject(list_plan_objs[plan_obj], outputFile, row)
                                        table_rows.append(row)
                                else:
                                    #print("is single plan")
                                    plan_obj = processSinglePlan(table)
                                    #outputFile.write("\"" + docxFileName + "\""+",True,False,True,")
                                    row['hasRateTable'] = "True"
                                    row['isMultiplan'] = "False"
                                    row['isNormalized'] = "True"
                                    row = standardizePlanObject(plan_obj, outputFile, row)
                                    table_rows.append(row)
                        if not rateTableFound:
                            #outputFile.write("\"" + docxFileName + "\""+",False,,,,,,,,,,\n")
                            row['hasRateTable'] = "False"
                            table_rows.append(row)
                    else:
                        #outputFile.write("\"" + docxFileName + "\""+",False,,,,,,,,,,\n")
                        row['hasRateTable'] = "False"
                        table_rows.append(row)
                else:
                    #outputFile.write("\""+file['fileName'] + "\""+",False,\n")
                    row['hasRateTable'] = "False"
                    table_rows.append(row)

            except Exception as e:
                print("Error collection rate table information: " + str(e))

        outputFile.close()
    errorFile.close()

    #for row in table_rows:
    #    print(row)

    return table_rows

#### Write out object from rateTableCSV() to an output file

In [207]:
def printRateTableCSV(table_rows):
    cwd = os.getcwd()

    errorFilePath = os.path.join(os.getcwd(),'data','cannot_process.csv')
    errorFile = open(errorFilePath, 'w')

    outputFilePath = os.path.join(os.getcwd(),'rate_table_output.csv')
    outputFile=open(outputFilePath, 'w')

    categoryList = ["Filename","hasRateTable","isMultiplan","isNormalized","DandP PPO","DandP non-PPO","basic PPO","basic non-PPO","major PPO","major non-PPO","orthodontic PPO","orthodontic non-PPO"]

    for row in table_rows:
        for cat in row:
            if not (cat in categoryList):
                categoryList.append(cat)

    #print(categoryList)
    for cat in categoryList:
        outputFile.write(cat + ",")
    outputFile.write("\n")

    for row in table_rows:
        for index,category in enumerate(categoryList):
            found = 0
            for attr in row:
                if attr == category:
                    outputFile.write("\""+row[attr]+"\"" + ",")
                    found = 1
            if not found:
                outputFile.write(",")
        outputFile.write("\n")


    outputFile.close()
    errorFile.close()    

#### A function for testing Rate Table extraction functions

In [208]:
def findingRateTablesTestFunction():
    from docx import Document
    cwd = os.getcwd()


    errorFilePath = os.path.join(os.getcwd(),'data','cannot_process.csv')
    errorFile = open(errorFilePath, 'w')


    base_info = None
    #pp = batchPreProcess(errorFile, os.path.join(cwd, "processed"))
    #if pp == None:
    #    print("Error in pp")
    #else:
    dataPath = os.path.join(cwd, "output")
    base_info = batchGetTokens(errorFile, dataPath)
    if not base_info:
        print("Error in getting tokens")
    else:
        outputFilePath = os.path.join(os.getcwd(),'rate_table_output.csv')
        outputFile=open(outputFilePath, 'w')
        first_row = 1

        outputFile.write("Filename,hasRateTable,isMultiplan,isNormalized,DandP PPO,DandP non-PPO,basic PPO,basic non-PPO,major PPO,major non-PPO,orthodontics PPO,orthodontics non-PPO\n")

        for file in base_info:#os.listdir(os.path.join(cwd, "processed")):
            count = 0
            try:
                file['fileName'] = os.path.split(file['filepath'])[1]
                if(isDenormalized(file['sentTokens'])):
                    outputFile.write("\""+file['fileName'] + "\"" + ",True,False,False,\n")

                elif not(file['fileName'].endswith("-pdf.txt")):
                    docxFileName = file['fileName'].replace("txt","docx")
                    document = Document(os.path.join(cwd, "processed", docxFileName))
                    docRateTables = []
                    print(docxFileName)#outputFile.write(file+",")
                    if(len(document.tables)>0):
                        rateTableFound = 0
                        for table in document.tables:
                            if(isRateTable(table)):
                                rateTableFound = 1
                                #print("is rate table")
                                if(isMultiPlan(table)):
                                    print("is Multiplan\n")
                                    list_plan_objs = processMultiPlan(table)
                                    print(len(list_plan_objs))
                                    for plan_obj in list_plan_objs:
                                        outputFile.write("\"" + docxFileName + "\"" + ",True,True,True,")
                                        standardizePlanObject(list_plan_objs[plan_obj], outputFile)
                                else:
                                    print("is single plan")
                                    plan_obj = processSinglePlan(table)
                                    outputFile.write("\"" + docxFileName + "\""+",True,False,True,")
                                    standardizePlanObject(plan_obj, outputFile)
                        if not rateTableFound:
                            outputFile.write("\"" + docxFileName + "\""+",False,,,,,,,,,,\n")
                    else:
                        outputFile.write("\"" + docxFileName + "\""+",False,,,,,,,,,,\n")
                else:
                    outputFile.write("\""+file['fileName'] + "\""+",False,\n")
            except Exception as e:
                print(str(e))

        outputFile.close()
    errorFile.close()

## Deductible Extraction
All functions so far for extracting deductible information from table objects.

#### Determine if table object is a Deductible table

In [ ]:
def isDeductibleTable(table):
    isDeductible = False
    keywords = ["deductible", "deductibles and maximum"]
    for kw in keywords:
        regex = re.compile(kw)
        if table.rows:
            for row in table.rows:
                for cell in row.cells:

                    results = re.search(regex, (cell.text).lower())
                    #print(results)
                    if results:
                        isDeductible = True
                    #print(cell.text)
        #if not isDeductible:



    return isDeductible

#### Process Deductible Table object        

In [210]:
def processDeductibleTable(table):

    tableInfo = {}
    categories = ["annual deductible","annual maximum","orthodontic maximum"]
    if isMultiPlan(table):
        print("is multiplan")
    else:
        first_row=1
        first_row_found = 0
        title_row=1
        cat_row = {}

        for row in table.rows:

            if(((row.cells[0]).text).lower().find("deductible") > -1) and not first_row_found:
                first_row_found = 1
            else:
                if(title_row):
                    ppoFlag = 0
                    non_ppoFlag = 0

                    ppoList = ["ppo providers", "pposm providers", "dpo providers", "delta dental ppo", "in-network"]
                    non_ppoList = ["non-delta dental providers","out-of-network"]

                    for cell in row.cells:
                        if any([x in (cell.text).lower() for x in ppoList]):
                            ppoFlag = 1
                        if any([x in (cell.text).lower() for x in non_ppoList]):
                            non_ppoFlag = 1
                    if ppoFlag and non_ppoFlag:

                        cat_row["categories"] = ""
                        for cell in range(1,len(row.cells)):
                            cat_row["categories"] = ";".join([cat_row["categories"], row.cells[cell].text])

                    title_row = 0

                if first_row_found:
                    new_row = {}
                    for cat in categories:
                        if cat in (row.cells[0].text).lower():
                            tableInfo[cat] = ""
                            for cell in range(1, len(row.cells)):
                                tableInfo[cat] = ";".join([tableInfo[cat], row.cells[cell].text])
                   
        plan = {}
        if(len(cat_row)>0):
            cats = cat_row["categories"].split(";")
            print(cats)
            for cat in cats:
                plan[cat] = {}
            for row in tableInfo:
                if row is "annual deductible":
                    cells = tableInfo[row].split(";")
                    moneyRegex = re.compile("\$\d+[,]?\d*")
                    for index, cell in enumerate(cells):
                        results = re.search(moneyRegex, cell)
                        if(results):
                            plan[cats[index]] = cell
        print(plan)
        
        return tableInfo

#### Function used to test deductible extraction

In [112]:
def findingDeductibleTablesTestFunction():
    from docx import Document
    cwd = os.getcwd()


    errorFilePath = os.path.join(os.getcwd(),'data','cannot_process.csv')
    errorFile = open(errorFilePath, 'w')


    base_info = None
    #pp = batchPreProcess(errorFile, os.path.join(cwd, "processed"))
    #if pp == None:
    #    print("Error in pp")
    #else:
    dataPath = os.path.join(cwd, "output")
    base_info = batchGetTokens(errorFile, dataPath)
    if not base_info:
        print("Error in getting tokens")
    else:
        #outputFilePath = os.path.join(os.getcwd(),'rate_table_output.csv')
        #outputFile=open(outputFilePath, 'w')
        #first_row = 1

        #outputFile.write("Filename,isRateTable,isMultiplan,isNormalized,DandP PPO,DandP non-PPO,basic PPO,basic non-PPO,major PPO,major non-PPO,orthodontics PPO,orthodontics non-PPO\n")

        for file in os.listdir(os.path.join(cwd, "processed")):
            #count = 0
            try:
                if(file.endswith(".docx")):
                    document = Document(os.path.join(cwd, "processed", file))
                    docRateTables = []
                    #print(file)#outputFile.write(file+",")
                    #print(file)
                    for table in document.tables:
                        if isDeductibleTable(table):#print(table)
                            print(file)
                            if(len(table.rows[0].cells[0].text)>0):
                                #print(file)
                                #print(table.rows[0].cells[0].text)
                                #print("\n")
                                deductibleInfo = processDeductibleTable(table)
                        #outputFile.write(file+",")
                        #if(isDeductibleTable(table)):
                        #    print(file)

            except Exception as e:
                print(str(e))

        #outputFile.close()
    errorFile.close()

## Run full workflow
Using all the functions, set up the workspace, process the files and extract metadata and rate table. 

#### This function is to be used after the output folder is filled with .txt files

In [211]:
def testOutputAttrCSV():
    cwd = os.getcwd()


    errorFilePath = os.path.join(os.getcwd(),'data','cannot_process.csv')
    errorFile = open(errorFilePath, 'w')


    base_info = None
    #pp = batchPreProcess(errorFile, os.path.join(cwd, "processed"))
    #if pp == None:
    #    print("Error in pp")
    #else:
    dataPath = os.path.join(cwd, "output")
    base_info = batchGetTokens(errorFile, dataPath)
    if not base_info:
        print("Error in getting tokens")
    else:
        outputFilePath = os.path.join(os.getcwd(),'data','raw_attribute_data.csv')
        outputFile=open(outputFilePath, 'w')
        first_row = 1
        for file in base_info:
            
            file_attr = getMetaDataAtt(file)
            #plans = getRateTables(file_attr)
            #if(plans and len(plans)>0):
            #    file_attr['hasRateTable'] = "True"
            #else:
            #    file_attr['hasRateTables'] = "False"

            try:
                filename = (os.path.split(file['filepath'])[1])
            except Exception as e:
                print(str(e))

            if first_row:
                for key in file_attr:
                    #print(key)
                    outputFile.write(key + ",")
                outputFile.write("\n")
                for attr in file_attr:
                    #print(attr + ": ")
                    attrStr = ""
                    if ((attr == 'fileTypes') or (attr == 'footer')) and file_attr[attr]:
                        for substr in file_attr[attr]:
                            attrStr = attrStr + substr + ";"
                        outputFile.write(attrStr+",")
                    else:
                        outputFile.write(str(file_attr[attr])+ ",")
                outputFile.write("\n")
                first_row = 0
            else:
                for attr in file_attr:
                    #print(attr + ": " + file_attr[attr])
                    attrStr = ""
                    if ((attr == 'fileTypes') or (attr == 'footer')) and file_attr[attr]:
                        for substr in file_attr[attr]:
                            attrStr = attrStr + substr + ";"
                        outputFile.write(attrStr+",")
                    else:
                        outputFile.write("\"" + str(file_attr[attr])+ "\",")
            #    outputFile.write(filename + ",")
            #    outputFile.write(str(count) + ",")
                outputFile.write("\n")#

        outputFile.close()
    errorFile.close()


#### This function assumes files are in the processed folder, but there does not have to be any .txt files created in the output folder

In [ ]:
def fullOutputAttrCSV():
    cwd = os.getcwd()


    errorFilePath = os.path.join(os.getcwd(),'data','cannot_process.csv')
    errorFile = open(errorFilePath, 'w')


    base_info = None
    pp = batchPreProcess(errorFile, os.path.join(cwd, "processed"))
    if pp == None:
        print("Error in pp")
    else:
        dataPath = os.path.join(cwd, "output")
        base_info = batchGetTokens(errorFile, dataPath)
        if not base_info:
            print("Error in getting tokens")
        else:
            outputFilePath = os.path.join(os.getcwd(),'data','raw_attribute_data.csv')
            outputFile=open(outputFilePath, 'w')
            first_row = 1
            for file in base_info:
                file_attr = getMetaDataAtt(file)
                plans = getRateTables(file_attr)
                if(plans and len(plans)>0):
                    file_attr['hasRateTable'] = "True"
                else:
                    file_attr['hasRateTables'] = "False"

                try:
                    filename = (os.path.split(file['filepath'])[1])
                    #print(filename)
        #
                except Exception as e:
                    print(str(e))###
    ##
    #
        #    #print(count)
                if first_row:
                    for key in file_attr:
                        #print(key)
                        outputFile.write(key + ",")
                    outputFile.write("\n")
                    for attr in file_attr:
                        #print(attr + ": ")
                        attrStr = ""
                        if ((attr == 'fileTypes') or (attr == 'footer')) and file_attr[attr]:
                            for substr in file_attr[attr]:
                                attrStr = attrStr + substr + ";"
                            outputFile.write(attrStr+",")
                        else:
                            outputFile.write(str(file_attr[attr])+ ",")
                    outputFile.write("\n")
                    first_row = 0
                else:
                    for attr in file_attr:
                        #print(attr + ": " + file_attr[attr])
                        attrStr = ""
                        if ((attr == 'fileTypes') or (attr == 'footer')) and file_attr[attr]:
                            for substr in file_attr[attr]:
                                attrStr = attrStr + substr + ";"
                            outputFile.write(attrStr+",")
                        else:
                            outputFile.write("\"" + str(file_attr[attr])+ "\",")
                #    outputFile.write(filename + ",")
                #    outputFile.write(str(count) + ",")
                    outputFile.write("\n")#

            outputFile.close()
    errorFile.close()

#### Run full process:

In [ ]:
def main(rawDataPath=None):
    rdp = ""    
    if rawDataPath is not None:
        rdp = rawDataPath
    elif(len(sys.argv)>1):
        rdp = sys.argv[1]
    else:
        print("Raw Data folder location required.")
        return
    
    outputPath = os.path.join(os.getcwd(), "output")
    dataPath = os.path.join(os.getcwd(),"data")
    processedPath = os.path.join(os.getcwd(),"processed")

    if(os.path.isdir(dataPath) or os.path.isdir(outputPath) or os.path.isdir(processedPath)):
        if(os.path.isdir(rdp)):
            clearData = input("This will overwrite current data, do you want to continue? [Y/N]\t")
            if clearData.strip().upper() == "Y":
                conf = input("Are you sure you want to overwrite current data? [Y/N]\t")
                if conf.strip().upper() == "Y":
                    try:
                        paths = []
                        paths.append(os.path.join(os.getcwd(), "processed"))
                        paths.append(os.path.join(os.getcwd(), "data"))
                        paths.append(os.path.join(os.getcwd(), "output"))

                        for path in paths:
                            shutil.rmtree(path)

                        rawToWorkspace(rdp)
                        fullOutputAttrCSV()
                        tableRows = rateTableCSV()
                        printRateTableCSV(tableRows)
                    except Exception as e:
                        print(str(e))
            elif clearData.strip().upper() == "N":
                print("Execution canceled.")
            else:
                print("incorrect input.")
        else:
            print("Path to raw data does not exist.")
    else:
        if(os.path.isdir(rdp)):
            rawToWorkspace(rdp)
            fullOutputAttrCSV()
            tableRows = rateTableCSV()
            printRateTableCSV(tableRows)
        else:
            print("Path to raw data does not exist.")

main("/Users/sydneyknox/Documents/data-insights/testDataRaw")


This will overwrite current data, do you want to continue? [Y/N]	Y
False
Are you sure you want to overwrite current data? [Y/N]	Y
cwd: /Users/sydneyknox/Documents/data-insights/data
creating /Users/sydneyknox/Documents/data-insights/data/processed...
creating /Users/sydneyknox/Documents/data-insights/data/output...
creating /Users/sydneyknox/Documents/data-insights/data/data...
cwd: /Users/sydneyknox/Documents/data-insights/data
processing: 19168 Contract.pdf
processing: 10041 Appendix C EOCc Meridian Union (Janb2014).pdf
processing: 10041 Appendix E EOC Daingerfield Union (Jan2018).pdf
processing: 19168 EOC.docx
processing: 01094 EOC  7-1-16.pdf
processing: 10041 Appendix F EOC Ardmore Union (Jan2012).doc
processing: 01094 - SI - Eff 7-1-17 to 6-30-19.doc
processing: 19168 Attachment B High.docx
processing: 01094 (SI) Eff. 7-1-15.doc
processing: 01094 (Tax Modfication).docx
processing: 01094 _SBC Modification_.pdf
processing: 1094 Full Contract (Eff. 7-1-07).pdf
processing: 10041 Appe

## Modification Tests
This is code that sets up and runs through the environments used in our tests on being able to modify original data.

In [ ]:
def modificationTest(pathToRawFile):
        import stat
        try:
            if os.path.isfile(pathToRawFile):

                dirPath = os.path.join(os.getcwd(),"modificationTestDir")
                if not os.path.isdir(dirPath):
                    os.makedirs(dirPath)

                        #print("moving: " + file)
                    shutil.copy(pathToRawFile, os.path.join(dirPath, os.path.split(pathToRawFile)[1]))

                    os.chmod(dirPath, stat.S_IRUSR | stat.S_IRGRP | stat.S_IROTH)
                    os.chmod(os.path.join(dirPath, os.path.split(pathToRawFile)[1]), stat.S_IRUSR | stat.S_IRGRP | stat.S_IROTH)
                    rawToWorkspace("C:\\Users\\Sydney.knox\\Documents\\data-insights\\modificationTestDir")
                    testOutputAttrCSV()
                    tableRows = rateTableCSV()
                    printRateTableCSV(tableRows)
                    #catch copy exception here so it doesn't stop all files (?)
            else:
                print("not a file")
        except Exception as e:
            print(str(e))

def modificationTest_changeFile(pathToRawFile):
        import stat
        try:
            if os.path.isfile(pathToRawFile):

                dirPath = os.path.join(os.getcwd(),"modificationTestDir")
                if not os.path.isdir(dirPath):
                    os.makedirs(dirPath)

                filepath = os.path.join(dirPath, os.path.split(pathToRawFile)[1])

                if not os.path.isfile(filepath):
                    shutil.copy(pathToRawFile, filepath)

                os.chmod(dirPath, stat.S_IRUSR | stat.S_IRGRP | stat.S_IROTH)
                os.chmod(filepath, stat.S_IRUSR | stat.S_IRGRP | stat.S_IROTH)

                word = win32.Dispatch("Word.application")
                #word = win32.gencache.EnsureDispatch('Word.Application')
                doc = word.Documents.Open(filepath)
                doc.Activate()

                # Rename path with .docx
                #new_file_abs = os.path.abspath(filepath)
                #new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)
                word.Selection.Find.Text = "the"
                word.Selection.Find.Replacement.Text = "Pariveda Solutions"
                word.Selection.Find.Execute(Replace=2, Forward=True)
                # Save and Close
                word.ActiveDocument.SaveAs(
                    filepath, FileFormat=16
                )

                word.Quit()
                    #catch copy exception here so it doesn't stop all files (?)
            else:
                print("not a file")
        except Exception as e:
            print(str(e))


#pathToRawFile = "C:\\Users\\Sydney.knox\\Documents\\rawDataDI\\TX 17404 Contract Regional (7.2.18).docx"
#modificationTest(pathToRawFile)
#modificationTest_changeFile(pathToRawFile)

## Services Extraction
This code was created by a separate Pariveda team that was on furlough for a couple days. It is the beginnings of services extraction, but needs work to be fully flexible and integrated with the rest of our codebase.

In [214]:
categories=['Diagnostic and Preventive Services', 'Basic Services', 'Major Services', 'Orthodontic Services']
planTypes=['PREVENTIVE PLAN', 'ENHANCED PLAN']
coverageNames=['Diagnostic:', 'Preventive:', 'Sealants:', 'Oral Surgery:', 'General Anesthesia or IV Sedation:', 'Endodontics:', 'Periodontics:', 'Palliative:', 'Restorative:', 'Specialist Consultations:', 'Night Guards/Occlusal Guard:', 'Night Guards/Occlusal Guards:', 'Crowns, Inlays/Onlays and Cast Restorations:', 'Other Basic Services:', 'Crowns and Inlays/Onlays:', 'Crowns and Inlays/Onlays and Gold Fillings:', 'Prosthodontics:', 'Denture Repairs:', 'Other Services:', 'Temporomandibular Joint (TMJ):', 'Other Major Services:']


def getFilesFromDir(path, extension):
    try:
        x = glob.glob(path + '/*.' + extension)
    except Exception as e:
        print(str(e))

    return x

def getFileBaseName(path, filename):
    return filename[len(path) + 1:]

#For the given docx file, search for the Contract Benefit Levels table and output the rows as a 2D array
def createPercentageTableList(docName):
    print('Printing table for ' + docName)
    document = Document(docName)
    percentages = []
    currPlan = 'DEFAULT'
    for i in range(len(document.tables)):
        table = document.tables[i]
        for x in range(len(table.rows)):
            row = table.rows[x]
            firstCell = row.cells[0].text.strip()
            if firstCell.upper() in planTypes:
                currPlan = firstCell.upper()
            if firstCell in categories:
                entry = [currPlan]
                for y in range(len(row.cells)):
                    cell = row.cells[y].text
                    entry.append(cell)
                percentages.append(entry)
    return percentages

def createDescriptionTableList(docName):
    print('Printing table for ' + docName)
    document = Document(docName)
    descriptions = []
    currPlan = 'DEFAULT'
    categoryCounter = 0
    incrementCounter = False
    for i in range(len(document.tables)):
        table = document.tables[i]
        if incrementCounter:
            categoryCounter += 1
            incrementCounter = False
        firstCell = table.rows[0].cells[0].text.strip()
        if firstCell.upper() in planTypes:
            currPlan = firstCell.upper()
            categoryCounter = 0
        elif isDescriptionTable(table):
            for x in range(len(table.rows)):
                row = table.rows[x]
                for y in range(len(row.cells)):
                    cell = row.cells[y].text.strip()
                    if cell in coverageNames and len(row.cells) > (y+1):
                        nextCell = row.cells[y+1].text.strip()
                        if nextCell not in coverageNames:
                            entry = [currPlan, categories[categoryCounter]]
                            incrementCounter = True
                            entry.append(cell[:-1])
                            entry.append(nextCell)
                            descriptions.append(entry)

    return descriptions

def isDescriptionTable(table):
    for x in range(len(table.rows)):
        row = table.rows[x]
        for y in range(len(row.cells)):
            cell = row.cells[y].text.strip()
            if cell in coverageNames:
                return True
    return False

def parseOrthodontics(filePath):
    document = Document(filePath)
    planIndex = 0
    orthodontics = []
    entry = []
    planList = getOrthodonticPlans(filePath)
    for x in range(len(document.paragraphs)):
        paragraph = document.paragraphs[x]
        if paragraph.text == 'Orthodontic Services' or paragraph.text == 'Orthodontic Benefits:':
            entry = [planList[planIndex], 'Orthodontic Services', '', document.paragraphs[x+1].text]
            orthodontics.append(entry)
            if (planIndex + 1) < len(planList):
                planIndex += 1
    return orthodontics

def multiplePlans(filePath):
    document = Document(filePath)
    for table in document.tables:
        firstCell = table.rows[0].cells[0].text.strip()
        if firstCell.upper() in planTypes:
            return True
    return False

def getOrthodonticPlans(filePath):
    document = Document(filePath)
    categoryIndex = 0
    planIndex = 0
    usedCategories = []
    planList = []
    if not multiplePlans(filePath):
        return ['DEFAULT']
    for paragraph in document.paragraphs:
        testCategory = paragraph.text.strip()
        if testCategory in categories:
            if testCategory not in usedCategories:
                usedCategories.append(testCategory)
            else:
                usedCategories = []
                planIndex += 1
            if testCategory == 'Orthodontic Services':
                planList.append(planTypes[planIndex])
    return planList

def main(path):
    try:
        files = getFilesFromDir(path, 'docx')

        with open('plans.tsv', 'w') as output:
            for file in files:
                filename = getFileBaseName(path, file)
                rows = createDescriptionTableList(file)
                ortho = parseOrthodontics(file)

                rows = rows + ortho

                for columns in rows:
                    columns.insert(0, filename)
                    rowString = '\t'.join(columns)
                    output.write(rowString + '\n')
    except Exception as e:
        print(str(e))

#main('/Users/sydneyknox/Documents/data-insights/data/raw')
